Each plant creates one part. For each part, determine a random number of materials it creates the part with. Then, generate a random list of distributors that provide that material.

In [1]:
import arcpy
import pandas as pd
import arcgis
from pprint import pprint
import random

In [2]:
plantFC = r"C:\Users\student\Documents\ArcGIS\Projects\KNOWDemo\SupplyChain.gdb\ProcessingPlants"
distFC = r"C:\Users\student\Documents\ArcGIS\Projects\KNOWDemo\SupplyChain.gdb\Distributors"

distSdf = pd.DataFrame.spatial.from_featureclass(distFC)
plantSdf = pd.DataFrame.spatial.from_featureclass(plantFC)

In [3]:
plantFields = ["Name", "Material", "Part", "Distributors"]
def addMatAndDist(inPlantFC, fields):
    matTypes = list(distSdf['Material'].unique())

    uCur = arcpy.da.UpdateCursor(inPlantFC, fields)

    def removeChars(arg1):
        strFromList = str(arg1)
        for char in ['[',']',"'"]:
            strFromList = strFromList.replace(char,"")
        return strFromList

    for row in uCur:
        num = int(random.choice([1,2,3,4]))
        mats = random.sample(matTypes, num)
        part = row[2]
        for mat in mats:
            distributorsList = list(distSdf.loc[distSdf['Material'] == mat]['Name'])
            
        matsStr = removeChars(mats)
        distList = removeChars(distributorsList)
            
        #print(f"Creating {part} from these {num} materials: {matsStr}. \nMaterials are from these distributors: {distList}")
        row[1] = matsStr
        row[3] = distList
        #print(row)
        #print()
        uCur.updateRow(row)
    #print("Done")
    del uCur

# use a search cursor to see if each distributor is in the list
def getUniqueDists(inPlantFC, field):
    src = arcpy.da.SearchCursor(inPlantFC, field)
    usedDists = []
    for row in src:
        for d in row[0].split(','):
            usedDists.append(d.strip())
    return(usedDists)

#for d in set(getUniqueDists(plantFC, plantFields)):


In [4]:
addMatAndDist(plantFC, plantFields)

d = set(getUniqueDists(distFC, "Name"))
p = set(getUniqueDists(plantFC, "Distributors"))

test = len(d) != len(p)

while d!= p:
    addMatAndDist(plantFC, plantFields)
    p = set(getUniqueDists(plantFC, "Distributors"))
    print(len(p))
    if d == p:
        print("They match")
        break

17
23
They match
